In [1]:
import os
import pandas as pd
from datetime import datetime
# from utils.charts import line_chart, hist_chart

In [ ]:

# 指定目录路径
dir_path = 'data/raw/sat3'

# 获取目录中的文件列表
file_list = os.listdir(dir_path)

# 加载目录中的CSV文件
csv_files = [f for f in file_list if f.endswith('.csv')]
dataframes = {}

for file in csv_files:
    file_path = os.path.join(dir_path, file)
    dataframes[file] = pd.read_csv(file_path)
    

# 处理每个CSV数据框，设置时间索引
for file, df in dataframes.items():
    # 添加列名（因为原始数据无表头）
    df.columns = ['timestamp', 'value']
    
    # 将时间戳列转换为datetime类型并设置为索引
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    
    # 更新dataframes字典
    dataframes[file] = df


In [ ]:
line_chart(dataframes)

In [ ]:
time_diffs={}
# 对每个dataframe绘制时间间隔分布图
for file, df in dataframes.items():
    # 计算时间间隔（毫秒）
    time_diffs[file] = df.index.to_series().diff().dt.total_seconds().dropna()
    # print(time_diffs.loc[time_diffs['timestamp']>50])
# time_diffs
hist_chart(time_diffs)

import panel as pn
import hvplot.pandas
import pandas as pd

# 初始化 Panel
pn.extension()

# 创建文件选择器控件
file_selector = pn.widgets.MultiSelect(
    name='选择数据文件',
    options=list(dataframes.keys()),
    value=[list(dataframes.keys())[0]],
    height=200
)

# 创建交互式函数
@pn.depends(files=file_selector)
def plot_intervals(files):
    if not files:
        return pn.pane.Markdown("请选择至少一个文件")
    
    df = dataframes[files[0]]
    time_diffs = df.index.to_series().diff().dt.total_seconds().dropna()
    time_diffs.columns = ['time_diff']
    print(time_diffs)
    
    # 创建直方图
    plot = time_diffs.hvplot.hist(
        time_diffs,
        bins=100,
        xlim=(14.25, 15.25),
        # logy=True,
        title='时间间隔分布',
        xlabel='时间间隔（秒）',
        ylabel='频率（对数刻度）',
        height=500,
        width=800,
    )
    return plot

# 创建交互式面板
dashboard = pn.Column(
    pn.pane.Markdown("## 数据采集时间间隔分析"),
    pn.Row(
        file_selector,
        plot_intervals
    )
)

# 显示面板
dashboard

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 设置中文字体和负号显示
plt.rcParams['font.sans-serif'] = ['Simhei']  # 显示中文
plt.rcParams['axes.unicode_minus'] = False    # 显示负号

# 对每个dataframe绘制数值间隔分布图
for file, df in dataframes.items():
    # 计算数值间隔（相邻值之间的差）
    value_diffs = df['value'].diff().dropna()
    
    # 创建新的图形
    plt.figure(figsize=(10, 6))
    
    # 绘制直方图
    plt.hist(value_diffs, bins=500)
    
    plt.yscale('log')
    
    # 设置标题和标签
    plt.title(f"文件 {file} 的数值间隔分布")
    plt.xlabel("数值间隔")
    plt.ylabel("频数")
    
    # 显示图形
    plt.tight_layout()
    plt.show()

In [ ]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt

# 获取第一个DataFrame的副本
first_file = list(dataframes.keys())[0]
df_copy = dataframes[first_file].copy()

# 提取value列数据作为观测值
observations = df_copy['value'].values

# 创建卡尔曼滤波器
kf = KalmanFilter(
    initial_state_mean=observations[0],
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.01,
    observation_covariance=1.0,
    initial_state_covariance=1.0
)

# 应用卡尔曼滤波
state_means, state_covs = kf.smooth(observations)

# 将滤波结果添加到DataFrame中
df_copy['kalman_filtered'] = state_means

# 绘制原始数据和滤波后的数据
plt.figure(figsize=(12, 6))
plt.plot(df_copy.index, df_copy['value'], 'b-', label='原始数据')
plt.plot(df_copy.index, df_copy['kalman_filtered'], 'r-', label='卡尔曼滤波后数据')
plt.title(f"文件 {first_file} 的卡尔曼滤波结果")
plt.xlabel("时间")
plt.ylabel("数值")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"原始数据和滤波后数据的前5行：")
print(df_copy.head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 设置中文字体和负号显示
plt.rcParams['font.sans-serif'] = ['Simhei']  # 显示中文
plt.rcParams['axes.unicode_minus'] = False    # 显示负号

# 计算数值间隔（相邻值之间的差）
value_diffs = df_copy['kalman_filtered'].diff().dropna()

# 创建新的图形
plt.figure(figsize=(10, 6))

# 绘制直方图
plt.hist(value_diffs, bins=500)

plt.yscale('log')

# 设置标题和标签
plt.title(f"文件 {file} 的数值间隔分布")
plt.xlabel("数值间隔")
plt.ylabel("频数")

# 显示图形
plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# 获取第一个数据框
file = list(dataframes.keys())[-1]
df_copy = dataframes[file].copy()
# print(df_copy.head())

# 应用Savitzky-Golay滤波
# 参数: 窗口长度=51, 多项式阶数=3
df_copy['sg_filtered'] = savgol_filter(df_copy['value'], window_length=20, polyorder=3)

print(df_copy.head(100))

# 绘制原始数据和滤波后的数据比较
plt.figure(figsize=(12, 6))
plt.plot(df_copy.index, df_copy['value'], 'b-', label='原始数据')
plt.plot(df_copy.index, df_copy['sg_filtered'], 'r-', label='滤波后数据')
plt.title(f"{first_file} 数据的Savitzky-Golay滤波")
plt.xlabel('时间')
plt.ylabel('数值')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 设置中文字体和负号显示
plt.rcParams['font.sans-serif'] = ['Simhei']  # 显示中文
plt.rcParams['axes.unicode_minus'] = False    # 显示负号

# 计算数值间隔（相邻值之间的差）
value_diffs = df_copy['sg_filtered'].diff().dropna()

# 创建新的图形
plt.figure(figsize=(10, 6))

# 绘制直方图
plt.hist(value_diffs, bins=500)

plt.yscale('log')

# 设置标题和标签
plt.title(f"文件 {file} 的数值间隔分布")
plt.xlabel("数值间隔")
plt.ylabel("频数")

# 显示图形
plt.tight_layout()
plt.show()

In [ ]:
import panel as pn
import hvplot.pandas

# 初始化 Panel
pn.extension()

# 创建文件选择器控件
file_selector = pn.widgets.Select(
    name='选择数据文件',
    options=list(dataframes.keys()),
    value=list(dataframes.keys())[0]
)

# 创建交互式函数
@pn.depends(file=file_selector)
def plot_data(file):
    df = dataframes[file]
    plot = df.hvplot.hist(
        y='value',
        # responsive=True,
        # downsample=True,
        # xlabel='时间',
        # ylabel='数值',
        height=500,
        bins=1000
    )
    return plot

# 创建交互式面板
dashboard = pn.Column(
    pn.pane.Markdown("## 数据可视化"),
    pn.Column(
        file_selector,
        plot_data
    )
)

# 显示面板
dashboard

In [ ]:
import panel as pn
import hvplot.pandas

# 初始化 Panel
pn.extension()

# 创建文件选择器控件
file_selector = pn.widgets.Select(
    name='选择数据文件',
    options=list(dataframes.keys()),
    value=list(dataframes.keys())[0]
)

colume_selector = pn.widgets.select(
    name='选择列',
    options=dataframes[file_selector].colume,
    value=dataframes[file_selector].colume
)

# 创建交互式函数
@pn.depends(file=file_selector, colume=colume_selector)
def plot_data(file):
    df = dataframes[file][colume]
    plot = df.hvplot.hist(
        y='value',
        # responsive=True,
        # downsample=True,
        # xlabel='时间',
        # ylabel='数值',
        height=500,
        bins=1000
    )
    return plot

# 创建交互式面板
dashboard = pn.Column(
    pn.pane.Markdown("## 数据可视化"),
    pn.Column(
        file_selector,
        plot_data
    )
)

# 显示面板
dashboard

In [ ]:
import pandas as pd
from collections import defaultdict

# 假设 'dataframes' 字典已在前面的单元格中定义并填充

# 按索引对 DataFrame 进行分组
# 注意：直接使用 pd.Index 作为字典键可能不可靠，转换为可哈希的类型（例如元组）
grouped_by_index = defaultdict(list)
for filename, df in dataframes.items():
    # 将索引转换为元组以便哈希和比较
    index_tuple = tuple(df.index)
    grouped_by_index[index_tuple].append(filename)

# 用于存储合并后的 DataFrame 的列表
merged_dataframes = []

# 合并每个组中的 DataFrame
for index_tuple, filenames in grouped_by_index.items():
    # 从原始字典中获取当前组的 DataFrame
    # 并将 'value' 列重命名为文件名
    dfs_to_merge = [dataframes[fname][['value']].rename(columns={'value': fname}) for fname in filenames]

    # 如果组中有 DataFrame，则合并它们
    if len(dfs_to_merge) > 0:
         # 沿列轴 (axis=1) 连接 DataFrame。由于索引相同，它们将正确对齐。
        merged_df = pd.concat(dfs_to_merge, axis=1)
        merged_dataframes.append(merged_df)

# 可选：显示第一个合并后的 DataFrame 作为示例，或显示所有合并后的 DataFrame 的信息
if merged_dataframes:
    print(f"共合并生成了 {len(merged_dataframes)} 个 DataFrame。")
    print("第一个合并后的 DataFrame 示例:")
    print(merged_dataframes[0].head())
    # 如果需要，可以取消注释以下行来查看所有合并后的 DataFrame
    for i, merged_df in enumerate(merged_dataframes):
        print(f"\\n--- Merged DataFrame {i+1} ---")
        print(merged_df.info())
        print(merged_df.head())
else:
    print("没有找到具有相同时间戳索引的 DataFrame 组进行合并。")